# Orogenic Landscape Modelling

We investigate the drainage network dynamics and the steady-state drainage patterns that emerge from erosion of an uplifting mountain.

In [1]:
import quagmire as qg

In [2]:
qg.nd = qg.scaling.non_dimensionalise

In [3]:
u = qg.scaling._scaling.u

In [4]:
scaling_coefficients = qg.scaling._scaling.get_coefficients()

In [5]:
scaling_coefficients["[length]"] = 80 * u.km
scaling_coefficients["[time]"] = 1000 * u.years

## Utilities

# Create Mesh

In [6]:
from quagmire import SurfaceProcessMesh
from quagmire import tools as meshtools
from quagmire import function as fn
from quagmire import equation_systems as systems
import quagmire
import numpy as np
import matplotlib.pyplot as plt
from time import time

%matplotlib inline

In [7]:
minX, maxX = 0.0, qg.nd(80. * u.km)
minY, maxY = 0.0, qg.nd(40. * u.km)
dx, dy = qg.nd(500 * u.m), qg.nd(500 * u.m)

x1, y1, simplices = meshtools.square_mesh(minX, maxX, minY, maxY, dx, dy, random_scale=1.0)
DM = meshtools.create_DMPlex(x1, y1, simplices, boundary_vertices=None)
mesh = SurfaceProcessMesh(DM, verbose=True, tree=True)

0 - Delaunay triangulation 0.02382347799721174s
0 - Calculate node weights and area 0.0013009700051043183s
0 - Find boundaries 0.0003773589851334691s
0 - cKDTree 0.001887164980871603s
0 - Construct neighbour cloud array 0.043750832992373034s
0 - Construct rbf weights 0.011492349993204698s


In [8]:
print( "\nNumber of points in the triangulation: {}".format(mesh.npoints))
print( "Downhill neighbour paths: {}".format(mesh.downhill_neighbours))


Number of points in the triangulation: 13041
Downhill neighbour paths: 2


In [9]:
boundary_mask_fn = fn.misc.levelset(mesh.mask)

### Initial topography

In [10]:
with mesh.deform_topography():
    mesh.topography.data = 0.

0 - Build downhill matrices 0.02269772201543674s
0 - Build upstream areas 0.0010379340092185885s


In [11]:
with mesh.deform_topography():
    new_elevation = qg.nd(100.*u.meter) * mesh.mask
    mesh.topography.data = new_elevation.evaluate(mesh)

0 - Build downhill matrices 0.01952939500915818s
0 - Build upstream areas 0.001286560989683494s


### Rainfall Function

In [12]:
rainfall_fn = mesh.add_variable(name="rainfall")
rainfall_fn.data = qg.nd(1.*u.m / u.year)

### Uplift function

In [13]:
uplift_rate_fn = mesh.add_variable(name="uplift")
uplift_rate_fn = qg.nd(1.0 * u.mm / u.year) * mesh.mask

### Stream Power Law

In [14]:
# vary these and visualise difference
m = fn.parameter(0.5)
n = fn.parameter(1.0)
K = fn.parameter(qg.nd(5.0e-6 / u.year))

# create stream power function
upstream_precipitation_integral_fn = mesh.upstream_integral_fn(rainfall_fn)
stream_power_fn = K*upstream_precipitation_integral_fn**m * mesh.slope**n * boundary_mask_fn

# evaluate on the mesh
sp = stream_power_fn.evaluate(mesh)

### Diffusion and Transport Solvers

In [15]:
import quagmire.equation_systems as systems

## Set up diffusion solver
diffusion_solver = systems.DiffusionEquation(mesh=mesh)
diffusion_solver.neumann_x_mask = fn.misc.levelset(mesh.mask, invert=True)
diffusion_solver.neumann_y_mask = fn.misc.levelset(mesh.mask, invert=True)
diffusion_solver.dirichlet_mask = fn.parameter(0.0)
diffusion_solver.diffusivity = fn.parameter(qg.nd(0.8 * u.m**2 / u.year))
diffusion_solver.verify() # Does nothing but is supposed to check we have everything necessary

# not needed to run
hillslope = diffusion_solver.phi
hillslope.data = mesh.topography.data

## Set up transport solver
transport_solver = systems.ErosionDepositionEquation(mesh=mesh, m=0.5, n=1.0)
transport_solver.rainfall = rainfall_fn
transport_solver.verify()

## Timestepping

In [16]:
mesh.verbose = False
save_fields = True

efficiency = fn.parameter(qg.nd(5.0e-6 / u.year))

h5_filename = "fields_{:06d}.h5"
stats = "Step {:04d} | dt {:.5f} | time {:.4f} | min/mean/max height {:.3f}/{:.3f}/{:.3f} | step walltime {:.3f}"
sim_time = 0.0
steps = 20

for i in range(steps):
    
    t = time()
    
    topography0 = mesh.topography.copy()
    
    # get timestep size   
    dt = min(diffusion_solver.diffusion_timestep(), transport_solver.erosion_deposition_timestep())
    
    # build diffusion, erosion + deposition
    diffusion_rate = diffusion_solver.diffusion_rate_fn(mesh.topography).evaluate(mesh)
    erosion_rate, deposition_rate = transport_solver.erosion_deposition_local_equilibrium(efficiency)
    uplift_rate = uplift_rate_fn.evaluate(mesh)
    dhdt = diffusion_rate - erosion_rate + uplift_rate #+ deposition_rate
    
    # do not rebuilt downhill matrix at half timestep
    mesh.topography.unlock()
    mesh.topography.data = mesh.topography.data + 0.5*dt*dhdt
    mesh.topography.lock()
    
    # get timestep size
    dt = min(diffusion_solver.diffusion_timestep(), transport_solver.erosion_deposition_timestep())
    
    # build diffusion, erosion + deposition
    diffusion_rate = diffusion_solver.diffusion_rate_fn(mesh.topography).evaluate(mesh)
    erosion_rate, deposition_rate = transport_solver.erosion_deposition_local_equilibrium(efficiency)
    uplift_rate = uplift_rate_fn.evaluate(mesh)
    dhdt = diffusion_rate - erosion_rate + uplift_rate#+ deposition_rate
    
    # now take full timestep
    with mesh.deform_topography():
        mesh.topography.data = topography0.data + dt*dhdt
        
    if save_fields:
        mesh.save_mesh_to_hdf5(h5_filename.format(i))
        mesh.save_field_to_hdf5(h5_filename.format(i), topo=mesh.topography.data)
        quagmire.tools.generate_xdmf(h5_filename.format(i))
        
    sim_time += dt
    
    if i/steps*100 in list(range(0,100,10)):
        topo_scaled = qg.scaling.dimensionalise(mesh.topography.data, u.meter)
        simulation_time = qg.scaling.dimensionalise(sim_time, u.year)
        print(stats.format(i, dt, simulation_time, topo_scaled.min(), topo_scaled.mean(),
                           topo_scaled.max(), time() - t))

Step 0000 | dt 1.64654 | time 1646.5372 year | min/mean/max height 0.011 meter/97.922 meter/102.727 meter | step walltime 0.445
Step 0002 | dt 1.64654 | time 4939.6116 year | min/mean/max height 0.032 meter/101.124 meter/108.189 meter | step walltime 0.484
Step 0004 | dt 1.64654 | time 8232.6861 year | min/mean/max height 0.051 meter/104.326 meter/113.661 meter | step walltime 0.474
Step 0006 | dt 1.64654 | time 11525.7605 year | min/mean/max height 0.068 meter/107.528 meter/119.143 meter | step walltime 0.492
Step 0008 | dt 1.64654 | time 14818.8349 year | min/mean/max height 0.083 meter/110.730 meter/124.635 meter | step walltime 0.436
Step 0010 | dt 1.64654 | time 18111.9093 year | min/mean/max height 0.096 meter/113.932 meter/130.135 meter | step walltime 0.481
Step 0012 | dt 1.64654 | time 21404.9838 year | min/mean/max height 0.107 meter/117.134 meter/135.643 meter | step walltime 0.447
Step 0014 | dt 1.64654 | time 24698.0582 year | min/mean/max height 0.117 meter/120.336 meter/